In [1]:
import requests

In [2]:
import glob

In [3]:
import json

In [4]:
import bs4 as bs

In [5]:
from bs4 import BeautifulSoup

In [6]:
import pandas as pd

In [7]:
##Generate empty list of PIDs
database_range = []

In [8]:
##Populate list of PIDs
database = pd.read_csv('/user/chjohnson1/for_elaine/cosmo_db.csv')
data = sorted(list(set(database['PROPOSID'].tolist())))
for element in data:
    if element >= 12795:
        database_range.append(element)
    else:
        pass

In [9]:
##Generate empty list to store URLs
database_files = []

In [10]:
##Populate list of URLs
for ID in database_range:
    if int(ID) > 12793:
        link = 'https://www.stsci.edu/hst/phase2-public/'+ str(ID) +'.apt'
        database_files.append(link)
    else:
        pass

In [11]:
##Dictionaries mapping database columns (PID and LINENUM) keys to Run ID values
PID_etcrunid_dict = {}
Linenum_etcrunid_dict = {}
rejects = []

In [71]:
##List of linenumbers
linenums = []
visitnumbers = []
for filenum in database_files:
    ##Retrieve proposal file from URL
    pagette = requests.get(filenum)
    ##Create BeautifulSoup object and parse file with xml parser
    objette = bs.BeautifulSoup(pagette.text, 'lxml')
    ##Generate list of visits
    visit_list = objette.find_all('visit')
    ##Iterate over each visit in visit list
    for vis in visit_list:
        ##Save visit number (first part of line number)
        visitnum = str(vis.attrs.get('number'))
        visitnumbers.append(visitnum)
        ##Iterate over each exposure in the visit
        exposure_list = vis.find_all('exposure')
        for expnum in exposure_list:
            ##Check if exposure uses FUV (filters out acq/nuv/mama exposures)
            configuration = expnum.attrs.get('config')
            if configuration == 'COS/FUV':
                ##Save exposure number (second part of line number)
                number = expnum.attrs.get('number')
                ##Line number formatting
                if len(str(number)) == 1:
                    exposurenum = '00' + str(number)
                else:
                    exposurenum = '0' + str(number)
                ##Assemble line number (visit + exposure)
                linenum = visitnum + '.' + exposurenum
                linenums.append(linenum)

In [84]:
PID_list = list(pd.read_csv('/users/ssunilkumar/documents/cosmo_db_modified_excel.csv')['PROPOSID'])[13067:52095]

39028

In [ ]:
##Get Run IDs for each rootname/PID
runids = []
for filenum in database_files:
    ##Retrieve file from URL
    pagette = requests.get(filenum)
    ##Create beautiful soup object
    objette = bs.BeautifulSoup(pagette.text, 'lxml')
    ##Find exposures
    exposure_list = objette.find_all('exposure')
    for exp in exposure_list:
        ##Filter out NUV/MAMA exposures
        if exp.attrs.get('config') == 'COS/FUV':
            ##Get Run ID
            etcrunid = exp.attrs.get('etcrunid')
            ##Fix IDs that don't have COS.sp. prefix
            if len(str(etcrunid)) == 6 or len(str(etcrunid)) == 7:
                complete_exposure = 'COS.sp.' + etcrunid
                runids.append(complete_exposure)
            ##Fix typo
            elif 'COS' not in str(etcrunid) and 'OS' in str(etcrunid):
                complete_exposure = 'C' + etcrunid
                runids.append(complete_exposure)
            ##Save IDs to list
            elif 'COS.sp.' or 'COS.sa.' or 'COS.ta.' or 'COS.im.' in etcrunid:
                if len(str(etcrunid)) == 13 or len(str(etcrunid)) == 14:
                    complete_exposure = etcrunid
                    runids.append(complete_exposure)
        else:
            pass

In [60]:
##IGNORE
linenums = []
runids = []
runid_linenum_dict = {}
for filenum in database_range[0:5]:
    pagette = requests.get('https://www.stsci.edu/hst/phase2-public/' + str(filenum) + '.apt')
    objette = bs.BeautifulSoup(pagette.text, 'lxml')
    visit_list = objette.find_all('visit')
    for vis in visit_list:
        exposure_list = vis.find_all('exposure')
        for exp in exposure_list:
            configuration = exp.attrs.get('config')
            etcrunid = exp.attrs.get('etcrunid')
            visnum = vis.attrs.get('number')
            if configuration == 'COS/FUV':
                exposurenum = exp.attrs.get('number')
                if len(exposurenum) == 1:
                    expnum = '00' + exposurenum
                    linenum = visnum + '.' + expnum
                    linenums.append(linenum)
                    runid = exp.attrs.get('etcrunid')
                    runids.append(runid)
                    runid_linenum_dict[linenum] = runid
                elif len(exposurenum) == 2:
                    expnum = '0' + exposurenum
                    linenum = visnum + '.' + expnum
                    linenums.append(linenum)
                    runid = exp.attrs.get('etcrunid')
                    runids.append(runid)
                    runid_linenum_dict[linenum] = runid
                else:
                    expnum = exposurenum
                    linenum = visnum + '.' + expnum
                    linenums.append(linenum)
                    runid = exp.attrs.get('etcrunid')
                    runids.append(runid)
                    runid_linenum_dict[linenum] = runid
            else:
                pass

In [90]:
##Write output to CSV
import csv
with open('/users/ssunilkumar/documents/cosmo_db_modified.csv', 'w') as f:
    writer = csv.writer(f)
    for val in runids:
        writer.writerow([val])